In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('air_system_previous_years.csv')

In [3]:
df2= pd.read_csv('air_system_present_year.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 171 entries, class to eg_000
dtypes: int64(1), object(170)
memory usage: 78.3+ MB


In [5]:
print(df)

      class  aa_000 ab_000      ac_000 ad_000 ae_000 af_000 ag_000 ag_001  \
0       neg   76698     na  2130706438    280      0      0      0      0   
1       neg   33058     na           0     na      0      0      0      0   
2       neg   41040     na         228    100      0      0      0      0   
3       neg      12      0          70     66      0     10      0      0   
4       neg   60874     na        1368    458      0      0      0      0   
...     ...     ...    ...         ...    ...    ...    ...    ...    ...   
59995   neg  153002     na         664    186      0      0      0      0   
59996   neg    2286     na  2130706538    224      0      0      0      0   
59997   neg     112      0  2130706432     18      0      0      0      0   
59998   neg   80292     na  2130706432    494      0      0      0      0   
59999   neg   40222     na         698    628      0      0      0      0   

      ag_002  ...   ee_002  ee_003   ee_004   ee_005   ee_006  ee_007  ee_0

In [13]:
df['class'].value_counts()

class
neg    59000
pos     1000
Name: count, dtype: int64

In [12]:
# Display basic DataFrame information
print("Info for df (air_system_previous_years.csv):")
df.info()

print("\nInfo for df2 (air_system_present_year.csv):")
df2.info()

# Print more detailed information for each column
print("\nDetailed column information for df:")
for column in df.columns:
    print(f"Column '{column}':")
    print(df[column].describe())
    print()

print("\nDetailed column information for df2:")
for column in df2.columns:
    print(f"Column '{column}':")
    print(df2[column].describe())
    print()

Info for df (air_system_previous_years.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 171 entries, class to eg_000
dtypes: int64(1), object(170)
memory usage: 78.3+ MB

Info for df2 (air_system_present_year.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Columns: 171 entries, class to eg_000
dtypes: int64(1), object(170)
memory usage: 20.9+ MB

Detailed column information for df:
Column 'class':
count     60000
unique        2
top         neg
freq      59000
Name: class, dtype: object

Column 'aa_000':
count    6.000000e+04
mean     5.933650e+04
std      1.454301e+05
min      0.000000e+00
25%      8.340000e+02
50%      3.077600e+04
75%      4.866800e+04
max      2.746564e+06
Name: aa_000, dtype: float64

Column 'ab_000':
count     60000
unique       30
top          na
freq      46329
Name: ab_000, dtype: object

Column 'ac_000':
count     60000
unique     2062
top           0
freq       8752
Name: ac_000, dty

In [16]:
# Replace 'na' with NaN
df.replace('na', np.nan, inplace=True)
df2.replace('na', np.nan, inplace=True)

# Identify the columns to be converted to float
columns_to_convert = df.columns.difference(['class'])

# Convert the columns to float
df[columns_to_convert] = df[columns_to_convert].astype(float)
df2[columns_to_convert] = df2[columns_to_convert].astype(float)

In [19]:
# Calculate the Pearson correlation matrix
cor_matr = df[columns_to_convert].corr(method='pearson')

# Find columns with high correlation >= 0.80 or <= -0.80
high_corr_pairs = []
low_corr_columns = set(columns_to_convert)

# Iterate over the correlation matrix to find pairs
for i in range(len(cor_matr.columns)):
    for j in range(i):
        if abs(cor_matr.iloc[i, j]) >= 0.80:
            colname1 = cor_matr.columns[i]
            colname2 = cor_matr.columns[j]
            high_corr_pairs.append((colname1, colname2, cor_matr.iloc[i, j]))
            low_corr_columns.discard(colname1)
            low_corr_columns.discard(colname2)

# Display the high correlation pairs
high_corr_pairs_df = pd.DataFrame(high_corr_pairs, columns=['Column1', 'Column2', 'Correlation'])
print("High Correlation Pairs:")
print(high_corr_pairs_df)

# Display the low correlation columns
low_corr_columns = list(low_corr_columns)
print("Low Correlation Columns:")
print(low_corr_columns)

High Correlation Pairs:
    Column1 Column2  Correlation
0    af_000  ae_000     0.823108
1    ag_004  ag_003     0.873726
2    ag_007  ag_006     0.856696
3    ah_000  aa_000     0.908103
4    ak_000  aj_000     0.930056
..      ...     ...          ...
411  ee_004  by_000     0.815882
412  ee_004  cn_004     0.877116
413  ee_004  cs_005     0.863802
414  ee_004  ee_002     0.885433
415  ee_004  ee_003     0.955882

[416 rows x 3 columns]
Low Correlation Columns:
['dy_000', 'dh_000', 'cs_008', 'ac_000', 'ef_000', 'ay_001', 'dj_000', 'dk_000', 'dd_000', 'az_002', 'ag_000', 'ee_008', 'cy_000', 'ax_000', 'bc_000', 'df_000', 'ee_005', 'ar_000', 'dg_000', 'cn_009', 'cn_008', 'ea_000', 'bz_000', 'cs_006', 'at_000', 'ch_000', 'ee_009', 'ct_000', 'ay_004', 'ag_001', 'cn_000', 'az_001', 'cs_009', 'cs_007', 'di_000', 'bs_000', 'as_000', 'cs_004', 'az_008', 'be_000', 'dq_000', 'cg_000', 'az_004', 'cl_000', 'au_000', 'cs_001', 'ee_006', 'eb_000', 'az_007', 'az_009', 'az_006', 'cj_000', 'de_000', 

In [20]:
len(low_corr_columns)
low_corr_columns_df = pd.DataFrame(low_corr_columns, columns=['LowCorrelationColumns'])

In [21]:
len(low_corr_columns)

83

In [31]:
# Prepare data for modeling
X = df[low_corr_columns]
y = df['class'].apply(lambda x: 1 if x == 'pos' else 0)  # Encode 'class' as 1 for 'pos' and 0 for 'neg'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('svd', TruncatedSVD(n_components=20)),  # Increase number of components
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=200))
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     11788
           1       0.75      0.60      0.67       212

    accuracy                           0.99     12000
   macro avg       0.87      0.80      0.83     12000
weighted avg       0.99      0.99      0.99     12000


In [32]:
# Use the model to make predictions on the present year data
X_present = df2[low_corr_columns]

# Apply the same preprocessing and transformation steps
X_present_imputed = pipeline.named_steps['imputer'].transform(X_present)
X_present_scaled = pipeline.named_steps['scaler'].transform(X_present_imputed)
X_present_svd = pipeline.named_steps['svd'].transform(X_present_scaled)

# Make predictions on the processed present year data
y_present_pred = pipeline.named_steps['xgb'].predict(X_present_svd)

# Add predictions to the present year data
df2['predicted_class'] = y_present_pred


In [33]:
df2['predicted_class'].value_counts()

predicted_class
0    15729
1      271
Name: count, dtype: int64